## 07 · Configure `TorchTrainer` with Ray Data  

Now we connect the Ray Dataset to the training loop using the `datasets` parameter in `TorchTrainer`:  

- **`datasets={"train": train_ds}`** → makes the transformed dataset available to the training loop as the `"train"` shard.  
- **`train_loop_ray_train_ray_data`** → the per-worker training loop that consumes Ray Data batches.  
- **`train_loop_config`** → passes hyperparameters (`num_epochs`, `global_batch_size`).  
- **`scaling_config`** → specifies the number of workers and GPUs to use (same as before).  
- **`run_config`** → defines storage for checkpoints and metrics.  

This setup allows Ray Train to automatically shard and stream the Ray Dataset into each worker during training.  

In [ ]:
# 07. Configure TorchTrainer with Ray Data integration

# Wrap Ray Dataset in a dict → accessible as "train" inside the training loop
datasets = {"train": train_ds}

trainer = TorchTrainer(
    train_loop_ray_train_ray_data,  # training loop consuming Ray Data
    train_loop_config={             # hyperparameters
        "num_epochs": 1,
        "global_batch_size": 512,
    },
    scaling_config=scaling_config,  # number of workers + GPU/CPU resources
    run_config=RunConfig(
        storage_path=storage_path, 
        name="dist-cifar-res18-ray-data"
    ),                              # where to store checkpoints/logs
    datasets=datasets,              # provide Ray Dataset shards to workers
)

### 08 · Launch Training with Ray Data  

Finally, call `trainer.fit()` to start the distributed training job.  

- Ray will automatically:  
  * Launch workers according to the `scaling_config`.  
  * Stream sharded, preprocessed batches from the Ray Dataset into each worker.  
  * Run the training loop (`train_loop_ray_train_ray_data`) on every worker in parallel.  
  * Report metrics and save checkpoints to the configured storage path.  

With this call, you now have a fully **end-to-end distributed pipeline** where **Ray Data handles ingestion + preprocessing** and **Ray Train handles multi-GPU training**.  

In [ ]:
# 08. Start the distributed training job with Ray Data integration

# Launches the training loop across all workers
# - Streams preprocessed Ray Dataset batches into each worker
# - Reports metrics and checkpoints to cluster storage
trainer.fit()